In [2]:
#use pol-stats environment

import pandas as pd
import numpy as np
import os
import cv2
from scipy.optimize import curve_fit
import colorcet as cc

import bokeh.io
import bokeh.plotting
import bokeh.models
import iqplot

from utils import force_calibration_params

bokeh.io.output_notebook()

Loading BokehJS ...

In [3]:
general_info_csv_filepath = '/Volumes/cytokinesis-zebrafish-collab/magnetic_tweezers_time_prec/2_analysis/time_prec_phases_tweezers_info.csv'
df_general_info = pd.read_csv(general_info_csv_filepath, delimiter=';', )

preliminary_plot_path = '/Volumes/cytokinesis-zebrafish-collab/magnetic_tweezers_time_prec/3_plots/preliminary/'

filepaths = df_general_info['trackmate_file'].values
tip_files = df_general_info['before_file'].values

calibration_params = {
            500: [5.59472213e+02, -4.17780556e+01,  1.61519352e+01, -8.84746192e+13],
            1000: [9.00468885e+02, -5.10553973e+01,  1.96828094e+01,  1.05746977e+14],
            2000: [6.47755226e+02, -5.16372481e+01, 1.87872313e+01, 4.49671461e+08]
        }


In [4]:
idx = 0
filepath=df_general_info['trackmate_file'].values[idx]
tip_file = df_general_info['before_file'].values[idx]

filename = os.path.basename(filepath).split('.')[0]  # This gets filename without extention

tip_threshold = df_general_info['tip_threshold'].values[idx]

first_pulse, t_on, t_off, dt,  = df_general_info[['first_pulse (frame)', 't_on (frame)', 't_off (frame)', 'time_interval (s)']].values[idx]

print(tip_threshold, first_pulse, t_on//dt, t_off//dt)

calibration_params = force_calibration_params[df_general_info['calibration (mV)'].values[idx]]

df = pd.read_csv(filepath, skiprows=[1, 2, 3]) # skiprows to get rid of the extensive header
df = df.sort_values(by='FRAME')

df.head(10)     #check out your df

550 12 120.0 360.0


,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,VISIBILITY,MANUAL_SPOT_COLOR,MEAN_INTENSITY_CH1,MEDIAN_INTENSITY_CH1,MIN_INTENSITY_CH1,MAX_INTENSITY_CH1,TOTAL_INTENSITY_CH1,STD_INTENSITY_CH1,CONTRAST_CH1,SNR_CH1
454,ID2002,2002,0,1.221630,469.850701,326.390423,0.0,0.0,0,6.0,1,NaN,422.205776,412.0,380.0,575.0,116951.0,30.974351,0.015849,0.425323
453,ID2001,2001,0,1.338763,469.791196,326.569429,0.0,0.5,1,6.0,1,NaN,421.501805,410.0,380.0,599.0,116756.0,34.774077,0.018790,0.447105
457,ID2005,2005,0,1.312828,469.850176,326.461673,0.0,1.0,2,6.0,1,NaN,420.682310,410.0,392.0,580.0,116529.0,33.776549,0.017548,0.429583
455,ID2003,2003,0,1.326064,469.752439,326.295689,0.0,1.5,3,6.0,1,NaN,419.974729,409.0,388.0,570.0,116333.0,34.131928,0.016783,0.406202
458,ID2006,2006,0,1.338973,469.737252,326.464264,0.0,2.0,4,6.0,1,NaN,421.386282,410.0,388.0,590.0,116724.0,33.168415,0.017613,0.439791
456,ID2004,2004,0,1.316895,469.830824,326.491654,0.0,2.5,5,6.0,1,NaN,420.693141,410.0,376.0,580.0,116532.0,33.355296,0.017440,0.432387
452,ID2000,2000,0,1.323186,469.929262,326.385629,0.0,3.0,6,6.0,1,NaN,420.635379,410.0,389.0,571.0,116516.0,31.784383,0.017016,0.442838
459,ID2007,2007,0,1.293867,469.892662,326.318428,0.0,3.5,7,6.0,1,NaN,420.577617,410.0,387.0,578.0,116500.0,33.616205,0.016153,0.397763
460,ID2008,2008,0,1.354242,469.885357,326.514491,0.0,4.0,8,6.0,1,NaN,420.480144,409.0,389.0,573.0,116473.0,33.720029,0.017604,0.431430
467,ID2015,2015,0,1.391168,469.884929,326.457773,0.0,4.5,9,6.0,1,NaN,421.924188,411.0,388.0,570.0,116873.0,34.611268,0.018715,0.447912


In [5]:
def find_tip(img: np.ndarray, threshold_tip: int = 750) -> list:
    tip_mask = img < threshold_tip
    tip_mask = np.array(tip_mask.astype(int))
    tip_outline = np.array([[0, 0]])
    tip_end = [0, 0]
    for i in range(1, tip_mask.shape[0]-1):
        for j in range(1, 300):
            if np.sum(tip_mask[i, j] != tip_mask[i-1:i+1, j-1:j+1])==2:
                tip_outline = np.concatenate([tip_outline, [[j, i]]], axis=0)
                if j > tip_end[0]:
                    tip_end = [j, i]
    if tip_outline.shape[0] > 20:
        return tip_outline, tip_end
    else:
        print("No tip found... try another threshold.")

def add_distance_from_tip(df: pd.DataFrame, tip_point: list[int, int]) -> None:
    '''
    calculates distance from the tip end point (tip_point) to bead and adds this to the dataframe df
    '''
    df['DISTANCE [um]'] = np.sqrt((df['POSITION_X']-tip_point[0])**2+(df['POSITION_Y']-tip_point[1])**2)


def add_force(df: pd.DataFrame, calibration_params: list) -> None:
    a1, k1, a2, k2 = calibration_params
    df['FORCE [pN]'] = a1*np.exp(df['DISTANCE [um]']/k1)+a2*np.exp(df['DISTANCE [um]']/k2)

In [11]:
import matplotlib.pyplot as plt
import numpy as np

cropped = True

save_to = '/Volumes/cytokinesis-zebrafish-collab/magnetic_tweezers_time_prec/3_plots/'

light_style = '/Users/ursic/PhD/Projects/1_Scripts/beads_in_zebrafish/presentation_style_light.mplstyle'
dark_style = '/Users/ursic/PhD/Projects/1_Scripts/beads_in_zebrafish/presentation_style_dark.mplstyle'
plt.style.use(light_style)


files_to_consider = ['20231212_s02p01t03_1_5sON_15sOFF_1000mV_1_SELECTED', '20231212_s01p01t06_1_5sON_25sOFF_1000mV_1']
track_idx_to_consider = [2, 0]
limits = np.array([[90, 250, 13.75, 17.25], [0, 160, 6.25, 9.75]])
if cropped:
    plt.rcParams["figure.figsize"] = (4/2, 3.5/2)
    limits = np.array([[150, 184, 15.5, 15.8], [60, 84, 7.5, 8.5]])

for (lim_id, track_idx, filename) in zip([1, 0], track_idx_to_consider, files_to_consider):
    for idx in range(len(df_general_info)):
        filepath = df_general_info['trackmate_file'].values[idx]
        tip_file = df_general_info['before_file'].values[idx]
        if os.path.basename(filepath).split('.')[0] == filename:
            break
            
    print(filename)
    df = pd.read_csv(filepath, skiprows=[1, 2, 3]) # skiprows to get rid of the extensive header
    df = df.sort_values(by='FRAME')

    # threshold before file to extract tip position, calculate tip shape
    tip_threshold = df_general_info['tip_threshold'].values[idx]
    img = cv2.imread(tip_file, cv2.IMREAD_UNCHANGED)
    tip_outline, tip_end = find_tip(img, tip_threshold)

    # tweezer data, imaging data
    first_pulse, t_on, t_off = map(int, df_general_info[['first_pulse (frame)', 't_on (frame)', 't_off (frame)']].values[idx])
    dt = df_general_info['time_interval (s)']

    # force calibration parameters
    calibration_params = force_calibration_params[df_general_info['calibration (mV)'].values[idx]]

    # calculations and expansion of the df
    add_distance_from_tip(df, tip_end)
    add_force(df, calibration_params)

    N_pulses = int(np.max(df['FRAME'])//(t_on+t_off)-1)
    magnet_pulses = np.array([[first_pulse + j*(t_on+t_off) + i - 1 for i in range(0, t_on)] for j in range(N_pulses+1)])
    df['MAGNET_STATUS'] = [1 if df['FRAME'].values[i] in magnet_pulses else 0 for i in range(len(df))]

    time_on = df.loc[(df["MAGNET_STATUS"]==1)&(df['TRACK_ID']==track_idx), 'POSITION_T'].values
    time_off = df.loc[(df["MAGNET_STATUS"]==0)&(df['TRACK_ID']==track_idx), 'POSITION_T'].values
    position_on = df.loc[(df["MAGNET_STATUS"]==1)&(df['TRACK_ID']==track_idx), 'DISTANCE [um]'].values /df.loc[(df["MAGNET_STATUS"]==1)&(df['TRACK_ID']==track_idx), 'FORCE [pN]'].values
    position_off = df.loc[(df["MAGNET_STATUS"]==0)&(df['TRACK_ID']==track_idx), 'DISTANCE [um]'].values /df.loc[(df["MAGNET_STATUS"]==0)&(df['TRACK_ID']==track_idx), 'FORCE [pN]'].values

    if not cropped:
        # plt.title('__phase')
        plt.xlabel('Time (s)')
        plt.ylabel('Bead position / force ($\mathrm{\mu}$m/pN)')

    plt.xlim(limits[lim_id, 0], limits[lim_id, 1])
    plt.ylim(limits[lim_id, 2], limits[lim_id, 3])
    plt.plot(time_on[:len(position_on)], position_on[:len(time_on)], alpha = 0.9, label = 'magnet ON', zorder=1)
    plt.plot(time_off[:len(position_off)], position_off[:len(time_off)], alpha =0.8, label = 'magnet OFF', zorder=0)
    
    

    if cropped:
        # plt.show()
        # plt.savefig(f'{save_to}{filename}_displacement_force_cropped_w.png')
        plt.savefig(f'{save_to}{filename}_displacement_force_cropped_w.svg', format='svg')
        plt.clf()
    else:
        plt.legend()
        # plt.show()
        # plt.savefig(f'{save_to}{filename}_displacement_force_w.png')
        plt.savefig(f'{save_to}{filename}_displacement_force_w.svg', format='svg')
        plt.clf()



20231212_s02p01t03_1_5sON_15sOFF_1000mV_1_SELECTED
20231212_s01p01t06_1_5sON_25sOFF_1000mV_1


<Figure size 600x525 with 0 Axes>